In [1]:
# *************************************************************************************
# *                          © 2019 Roshini Saravanakumar                             *
# *************************************************************************************

import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter

df = pd.read_csv('../data/raw_data.csv')
df.drop(columns='Row Number', inplace=True)

df = df[['Area', 'Item', 'Year', 'Production', 'Import Quantity', 'Export Quantity']]
df.rename(columns={'Area': 'Country', 'Item':'Produce', 'Production':'Production Quantity'}, inplace=True)

# if both production and import quantity is 0, this data is not meaningful
df = df[df['Production Quantity'] + df['Import Quantity'] > 0]
df.head() 

,Country,Produce,Year,Production Quantity,Import Quantity,Export Quantity
0,Armenia,Potatoes,2000,290260.0,390.0,300.0
1,Armenia,Potatoes,2001,363834.0,2290.0,0.0
2,Armenia,Potatoes,2002,374263.0,1918.0,0.0
3,Armenia,Potatoes,2003,507518.0,2853.0,7.0
4,Armenia,Potatoes,2004,576427.0,1553.0,0.0


In [2]:
def calculateConsumption(entry):
    return entry['Production Quantity'] + entry['Import Quantity'] - entry['Export Quantity']

# calculate the consumption for each entry (produce + import) - export
df['Consumption Quantity'] = df.apply(calculateConsumption, axis=1)

In [3]:
def calculatePercentConsumed(entry):
    return (entry['Consumption Quantity'] / (entry['Production Quantity'] + entry['Import Quantity'])) * 100

# calculate the percentage of the total yield(production + import) for a crop that was consumed
df['Percent Consumed'] = df.apply(calculatePercentConsumed, axis=1)

In [4]:
def calculatePercentExported(entry):
    return (entry['Export Quantity'] / (entry['Production Quantity'] + entry['Import Quantity'])) * 100

# calculate the percentage of the total yield(production + import) for a crop that was exported
df['Percent Exported'] = df.apply(calculatePercentExported, axis=1)

In [5]:
# filter out entries that do not have complete info (more was exported than what was produced + imported)
df = df[(df['Percent Consumed'] >= 0) & (df['Percent Exported'] >= 0)]
df.sample(n=5)

,Country,Produce,Year,Production Quantity,Import Quantity,Export Quantity,Consumption Quantity,Percent Consumed,Percent Exported
138040,Land Locked Developing Countries,Currants,2006,3922.0,72.0,5.0,3989.0,99.874812,0.125188
8079,Kazakhstan,Potatoes,2006,2361620.0,34874.0,73.0,2396421.0,99.996954,0.003046
137101,Land Locked Developing Countries,Vetches,2012,328941.0,0.0,0.0,328941.0,100.000000,0.000000
40287,Portugal,Spinach,2008,14849.0,222.0,1449.0,13622.0,90.385509,9.614491
146521,Finland,Cauliflowers and broccoli,2013,4691.0,3088.0,1.0,7778.0,99.987145,0.012855


In [6]:
# calculate ranks(1 -> n) for each crop produced by a given country in a given year 
# based on production quantity (ex/ rank 1 = crop with highest production quantity) 
df = df[['Country', 'Produce', 'Year', 'Production Quantity', 'Percent Consumed']]
df['Rank'] = df.groupby(by=['Country', 'Year'])['Production Quantity'].rank(ascending=False, method='first')

# sort the values by rank with highest ranks appearing at the top so
# when we calculate cumulative maximum of each group, we get the size of the group
df.sort_values('Rank', ascending=False, inplace=True)
df['Group Size'] = df.groupby(by=['Country', 'Year'])['Rank'].cummax()

In [7]:
# testcase to ensure this works
def testRank(dataframe):
    # sort values by descending rank, ascending production quantity
    dataframe.sort_values('Rank', ascending=False)
    lastProductionQuantity = 0
    for index, row in dataframe.iterrows():
        if(float(row['Group Size']) != dataframe.index.size):
            print(str(row['Group Size']) + ' != ' + str(dataframe.size))
            return 'FAIL'
        if row['Production Quantity'] < lastProductionQuantity:
            return 'FAIL'
        else:
            lastProductionQuantity = row['Production Quantity']
    return 'PASS'
    
testRank(df[(df['Country'] == 'Costa Rica') & (df['Year'] == 2010)])

'PASS'

In [8]:
def calculateScaledPercentConsumed(entry):
    return entry['Percent Consumed'] * (entry['Rank'] / entry['Group Size'])

# calculate a scaled percent consumption for each entry that can be used to determine which crops
# have both high production AND low percent consumption
df['Scaled Percent Consumed'] = df.apply(calculateScaledPercentConsumed, axis=1)

In [21]:
# This is the top producer of coconuts in 2014. here the scale has a GREAT effect on the 
# percent consumption, by decreasing it drastically 
df[(df['Produce'] == 'Coconuts') & (df['Year'] == 2014)].nlargest(1, 'Production Quantity')

,Country,Produce,Year,Production Quantity,Percent Consumed,Rank,Group Size,Scaled Percent Consumed
91587,World,Coconuts,2014,60987911.0,98.586075,21.0,113.0,18.321306


In [22]:
# This is the lowest producer of coconuts in 2014, as you can see the scale has 
# virtually no effect on the percent consumption (percent consumed = scaled percent consumed)
df[(df['Produce'] == 'Coconuts') & (df['Year'] == 2014)].nsmallest(1, 'Production Quantity')

,Country,Produce,Year,Production Quantity,Percent Consumed,Rank,Group Size,Scaled Percent Consumed
72642,British Virgin Islands,Coconuts,2014,50.0,53.703704,2.0,2.0,53.703704
